In [ ]:
import torch
from src.k_tree import Ktree
from src.utils.data import loadData_3d

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data, _ = loadData_3d(1000, 100)

learning_rate = 5e-3 # 1e-2
teacher_args = {
    "optimizer_lr": learning_rate,
    "epochs": 100, # 200
    "times": 10,
    "alpha": 10, # 5
    "beta": 5
}
un_args = {
    "N": 30,
    "M": 30*30-1, # 10000 - 1
    "epsilon": 0.14 # here a layer value should be somehow included. As layers get denser  epilson should be smaller
}
student_args = {
    "optimizer_lr": 5e-3,
    "epochs": 2000, # 30000
}

threshold = 0.1 * len(data)
ktree = Ktree(threshold, data, teacher_args, un_args, student_args, 3)
ktree.create_tree(save_path_prefix="models/cuboids/1000/class")

In [ ]:
# Print a student prediction both using the student model from training
# and the student model loaded from config. The two outputs should be equal.
query = torch.tensor([[4., 2., 0.], [2., 0., -2.]]).to(device)

pred_trained = ktree.root.student(query)
print(pred_trained)

# Recreate the student model from config.
node = Ktree.Node(ktree.root.data, "", ktree, None)
node.create_student_from_config("models/cuboids/1000/class0_student_config.pt")
pred_config = node.student(query)
print(pred_config)

print("Predictions are " + "equal." if torch.equal(pred_trained, pred_config) else "NOT equal.")

In [ ]:
import numpy as np

nop =  8 ** 3  # 512 points.
random_p = torch.zeros(nop, 3)
x_lim = [50, 950]
y_lim = [50, 950]
z_lim = [50, 950]
base = 8
for i, x in enumerate(np.linspace(x_lim[0],x_lim[1], base)):
    for j, y in enumerate(np.linspace(y_lim[0],y_lim[1], base)):
        for k, z in enumerate(np.linspace(z_lim[0],z_lim[1], base)):
            index = i * (base ** 2) + j * base + k
            random_p[index] = torch.Tensor([x, y, z])
 
infer_points = random_p.to(device)
num_of_correct_predictions = 0
for query_point in infer_points:
    pred = ktree.query(query_point)
    brute_force = ktree.root.query(query_point)
    if np.array_equal(pred, brute_force):
        num_of_correct_predictions += 1
 
print(f"The percentage of correct predictions is {(num_of_correct_predictions / len(infer_points)) * 100:.2f}%")

In [ ]:
acc = 0
mean_acc = 0
times = 0

while times < 4:
    # Now let's create randomly N points in the space 50-950, 50-950, 50-950.
    n = 1000
    random_p = torch.zeros(n, 3)
    bounding_box = [[50, 950]] * 3
    
    for i in range(n):
        random_p[i] = torch.Tensor([np.random.randint(*area) for area in bounding_box])
    infer_points = random_p.to(device)

    num_of_correct_predictions = 0
    for query_point in infer_points:
        pred = ktree.query(query_point)
        brute_force = ktree.root.query(query_point)
        if np.array_equal(pred, brute_force):
            num_of_correct_predictions += 1

    acc = num_of_correct_predictions / len(infer_points) * 100
    mean_acc += acc
    times += 1
    print(f"The percentage of correct predictions (loop {times}) is {acc:.2f}%")

print(f"The best percentage of correct predictions is {acc:.2f}%")
print(f"The mean percentage of correct predictions is {mean_acc/times:.2f}%")